#### ingest a set of multiline files in qualifying folder

- read json file
- rename columns and add new columns
- write output to silver container

In [0]:
from pyspark.sql.functions import current_timestamp, col, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
qualifying_schema = StructType([
    StructField("qualifyingId", IntergerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("contructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("q1", StringType(), True),
    StructField("q2", StringType(), True),
    StructField("q3", StringType(), True)
])

In [0]:
qualifying_df = spark.read \
    .schema(pit_stops_schema)\
    .option("multiline", "true")\
    .json("abfss://bronze@motorracingdl.dfs.core.windows.net/qualifying")

##### rename columns and add columns

In [0]:
qualifying_final_df = qualifying_df.withColumnRenamed("qualifyingId", "qualifying_id")\
                                 .withColumnRenamed("driverId", "driver_id")
                                 .withColumnRenamed("raceId", "race_id")\
                                 .withColumnRenamed("constructorId", "constructor_id")
                                 .withColumn("ingestion_date", current_timestamp())


In [0]:
# qualifying_final_df.write.mode("overwrite").parquet("abfss://bronze@motorracingdl.dfs.core.windows.net/")